## European Medicine Agency (EMA)

The European Medicines Agency (EMA) is a decentralized agency of the European Union (EU) responsible for the scientific evaluation, supervision, and safety monitoring of medicines in the EU. EMA plays a crucial role in the public health domain by ensuring that all medicines available on the EU market are safe, effective, and of high quality.

## Summary of product characteristics (SmPC)

A document describing the properties and the officially approved conditions of use of a medicine. Summaries of product characteristics form the basis of information for healthcare professionals on how to use the medicine safely and effectively. Abbreviated as SmPC. 

<br/>

### Structure of an SmPC
The structure of the SmPC is defined by the European pharmaceutical legislation. The information included in the SmPC should be product specific and can be cross-referenced to avoid any redundancy. It should be documented in a clear language and should not lead to any ambiguity. The SmPC is divided into 6 major sections:

1. Name of the product
2. Composition
3. Pharmaceutical Form
4. Clinical particulars – Includes therapeutic indications, recommendation for dosages and safety information
5. Pharmacological properties – Takes into account the therapeutic indications of the clinical elements and their potential adverse drug reactions
6. Pharmaceutical particulars – Includes Regulatory information related to the drug

<br/>

<img src="./images/structureSmPC.png" width="600px;">

<br/>
<br/>

### Prescribing medicines for special populations
When prescribing medication to specific groups like children, the elderly, pregnant and breastfeeding women, and individuals with kidney or liver issues, it's
crucial to exercise caution. The public documentation for medicines, especially the Summary of Product Characteristics (SmPC) and the package leaflets (PLs), often includes many of these essential precautions.
<br/>

<img src="./images/SmPC_authorization_process.png" width="800px;">

When seeking marketing authorization, a company submits an application dossier that contains instructions for healthcare professionals on the safe and effective use of the medicine. In Europe, this is referred to as the Summary of Product Characteristics (SmPC). The SmPC must be regularly updated over the medicine's lifecycle to reflect new data on efficacy or safety. The figure above illustrates the connection between a medicine's development, the regulatory dossier with the initial proposed SmPC, the SmPC that gets approved, and subsequent updates to the SmPC.


[1]: https://www.ema.europa.eu/en/glossary/summary-product-characteristics
[2]: https://www.freyrsolutions.com/what-is-an-smpc


### Project Overview

Using natural language processing (NLP) techniquesto automatically extract adverse drug reactions from such unstructured textual information helps clinicalexperts to effectively and efficiently use them in daily practices. Such techniques have been developed for Structured Product Labels from the Food and Drug Administration (FDA), but there is no researchfocusing on extracting from the Summary of Product Characteristics.


In this work, we built a naturallanguage processing pipeline that automatically scrapes the summary of product characteristics onlineand then extracts adverse drug reactions from them.

<img src="./images/Project_Overview.png" width="300px;">

##  STEP 1: Download medicine data and import them to Database
You can download the European Medicines Agency's (EMA) medicine-related data published on this website in Excel table format from this page. EMA updates these medicine data tables once a day. The following two files need to be downloaded:

<br/>

* The file listing all medicines, **including the withdrawn and not authorised ones**, named medicines_output_european_public_assessment_reports_en.xlsx, found [here](https://www.ema.europa.eu/sites/default/files/Medicines_output_european_public_assessment_reports.xlsx)

and

* The file listing **all the URLs for the SmPCs**, named Product information URLs for member product-information-urls-member-states_en.xls, found [here](https://www.ema.europa.eu/documents/product-information/product-information-urls-member-states_en.xls)


##### Disclaimer
The database is used purely to join the two csv. The view can also be done with dataframes without the need to use the database

[3]: https://toolbox.eupati.eu/resources/prescribing-medicines-for-special-populations/?print=print
[4]: https://www.ema.europa.eu/en/homepage


In [1]:
import os
import pandas as pd
import re
import sqlite3
import camelot

# Define the folder that you store the required xls
PATH = "./data"

MEDICINE_LIST = "medicines_output_european_public_assessment_reports_en.xlsx"
PRODUCT_URLS_LIST = "product-information-urls-member-states_en.xls"

In [2]:
medicinesOutputData = pd.read_excel(os.path.join(PATH, MEDICINE_LIST))
medicinesOutputData.head()

,Category,Medicine name,Therapeutic area,International non-proprietary name (INN) / common name,Active substance,Product number,Patient safety,Authorisation status,ATC code,Additional monitoring,...,Vet pharmacotherapeutic group,Date of opinion,Decision date,Revision number,Condition / indication,Species,ATCvet code,First published,Revision date,URL
0,Human,Ryeqo,Leiomyoma,"relugolix, estradiol, norethisterone acetate","Relugolix, norethisterone acetate, estradiol h...",EMEA/H/C/005267,no,Authorised,H01CC54,yes,...,NaN,2021-07-16 00:00:00,2023-10-30 01:00:00,6.0,Ryeqo is indicated in adult women of reproduct...,NaN,NaN,2021-05-19 14:14:00,2023-12-04 18:41:00,https://www.ema.europa.eu/en/medicines/human/E...
1,Human,Yesafili,Macular Edema; Retinal Vein Occlusion; Diabe...,aflibercept,aflibercept,EMEA/H/C/006022,no,Authorised,S01LA05,yes,...,NaN,2023-07-20 00:00:00,2023-11-29 01:00:00,1.0,Yesafili is indicated for adults for the treat...,NaN,NaN,2023-07-17 16:27:00,2023-12-04 18:15:00,https://www.ema.europa.eu/en/medicines/human/E...
2,Human,Leflunomide medac,"Arthritis, Rheumatoid",leflunomide,leflunomide,EMEA/H/C/001227,no,Authorised,L04AA13,no,...,NaN,NaT,2023-10-31 01:00:00,19.0,Leflunomide is indicated for the treatment of ...,NaN,NaN,2018-01-30 14:10:00,2023-12-04 17:56:00,https://www.ema.europa.eu/en/medicines/human/E...
3,Human,Filgrastim Hexal,Neutropenia; Hematopoietic Stem Cell Transpla...,filgrastim,filgrastim,EMEA/H/C/000918,no,Authorised,L03AA02,no,...,NaN,NaT,2023-11-30 01:00:00,24.0,Reduction in the duration of neutropenia and t...,NaN,NaN,2018-05-24 17:15:00,2023-12-04 17:43:00,https://www.ema.europa.eu/en/medicines/human/E...
4,Human,Opzelura,Vitiligo,ruxolitinib,ruxolitinib phosphate,EMEA/H/C/005843,no,Authorised,D11AH,no,...,NaN,2023-02-23 01:00:00,2023-10-19 00:00:00,2.0,Opzelura is indicated for the treatment of non...,NaN,NaN,2023-02-22 15:05:00,2023-12-04 17:35:00,https://www.ema.europa.eu/en/medicines/human/E...


In [3]:
productInformationURLs = pd.read_excel(os.path.join(PATH, PRODUCT_URLS_LIST))
productInformationURLs.head()

,ProductName,ProductNumber,Domain,LanguageCode,URL(oldwebsite),URL(currentwebsite)
0,Abasaglar (previouslyAbasria),EMEA/H/C/002835,human,en,http://www.ema.europa.eu/docs/en_GB/document_l...,https://www.ema.europa.eu/en/documents/product...
1,Abecma,EMEA/H/C/004662,human,en,NaN,https://www.ema.europa.eu/en/documents/product...
2,Abevmy,EMEA/H/C/005327,human,en,NaN,https://www.ema.europa.eu/en/documents/product...
3,Abilify,EMEA/H/C/000471,human,en,http://www.ema.europa.eu/docs/en_GB/document_l...,https://www.ema.europa.eu/en/documents/product...
4,Abilify Maintena,EMEA/H/C/002755,human,en,http://www.ema.europa.eu/docs/en_GB/document_l...,https://www.ema.europa.eu/en/documents/product...


In [4]:
conn = sqlite3.connect(os.path.join(PATH, "SmPCs.db"))

medicinesOutputData.to_sql('emaMedicinesOP', conn, if_exists='replace', index=False)
productInformationURLs.to_sql('emaProductInformationURLs', conn, if_exists='replace', index=False)

1928

## Step 2: Create csv with SmPCs URLs

 The actual SmPCs information will be extracted from the joinedHumanAuthorisedProductURLsONLY.csv file which contains the URLs for the authorised medicines for human use.

In [ ]:
joinedHumanAuthorisedProductURLsONLY = pd.read_sql("""
    SELECT [URL(currentwebsite)]
    FROM emaMedicinesOP
    LEFT JOIN emaProductInformationURLs 
    ON emaMedicinesOP.[Medicine name]=emaProductInformationURLs.[ProductName]
    WHERE  emaMedicinesOP.Category='Human'
      AND emaMedicinesOP.[Authorisation Status] ='Authorised';
""", conn)

joinedHumanAuthorisedProductURLsONLY.to_csv(os.path.join(PATH, "joinedHumanAuthorisedProductURLsONLY.csv"))

## Step 3: Download SmPCs pdfs


In [ ]:
import requests

In [ ]:
urlsDF = pd.read_csv(os.path.join(PATH, "joinedHumanAuthorisedProductURLsONLY.csv"))
urlsDF.dropna(inplace = True)
urls = urlsDF['URL(currentwebsite)'].tolist()

In [ ]:
for index, url in enumerate(urls):
    response = requests.get(url)
    file_name = url.split('/')[-1]

    print("Download ", index, " with name: ", file_name)

    if response.status_code == 200:
        with open(os.path.join(PATH, "pdfs", file_name), 'wb') as pdf_file:
            pdf_file.write(response.content)
    else:
        print(f"Failed to download the : {file_name}")

### Step 4: Extract Tabulated Summary of Adverse Reactions from the 4.8 section 

This section should include all adverse reactions for which, after thorough assessment, a causal relationship between the medicinal product and the adverse event is at least a reasonable possibility. These ADRs should be all reactions from:
* Clinical trials
* Post-authorisation safety studies
* Spontaneous reporting

#####  The tabulated list of adverse reactions in section 4.8:
* Introduce table with short paragraph stating source of safety database
* Separate tables are acceptable in exceptional cases where adverse profiles markedly differ depending on the use of the product.

#####  Structure of the table: general considerations

* Table should be written according to MedDRA system organ classification (SOC)
* A pragmatic approach to the location of terms should be taken in order to make the identification of adverse reactions simpler and clinically appropriate for the reader
* Within each SOC, adverse reactions should be ranked under headings of frequency, most frequent reactions first
* Frequency grouping, adverse reactions should be presented in order of decreasing seriousness

##### Frequency Grouping:
* Very common (≥1/10)
* Common (≥1/100 to <1/10)
* Uncommon (≥1/1,000 to <1/100)
* Rare (≥1/10,000 to <1/1,000)
* Very rare (<1/10,000)
* Frequency not known (cannot be estimated from the available data)

#### SmPCs Examples

<img src="./images/SmPCs_example.png" width="500px;">


## MedDRA
MedDRA is a multilingual terminology allowing most users to operate in their native languages. The table below identifies the initial MedDRA version when each MedDRA language was made available to users. 

<br />

<img src="./images/MedDRA_Hierarchy.png" width="500px;">

<br />

The most important reason to “code” data into a standardised terminology is to analyse it. A key benefit of MedDRA is in its support of straightforward as well as sophisticated analyses. MedDRA can be used to analyse individual medical events (e.g., “Influenza”) or issues involving a system, organ or aetiology (e.g., infections) using its hierarchical structure. MedDRA can be used for signal detection and monitoring of clinical syndromes whose symptoms encompass numerous systems or organs using its multiaxial hierarchy or through the special feature of Standardised MedDRA Queries.

<br />

### MedDRA Hierarchy

The structure of MedDRA is very logical. There are five levels to the MedDRA hierarchy, arranged from very specific to very general.

<br />

Download [MedDRA codes](https://www.ema.europa.eu/en/documents/other/meddra-important-medical-event-terms-list-version-261_en.xlsx) and extract the unique values from the SOC Name column

In [5]:
medraSOC = pd.read_excel(os.path.join(PATH, "MedDRA", "meddra-important-medical-event-terms-list-version-261_en.xlsx"))
socList = medraSOC['SOC Name'].unique()
SOCListRegex = '(?:% s)' % '|'.join(socList)
regex = re.compile(SOCListRegex)

In [6]:
regex

re.compile(r'(?:Blood and lymphatic system disorders|Cardiac disorders|Congenital, familial and genetic disorders|Endocrine disorders|Eye disorders|Gastrointestinal disorders|Hepatobiliary disorders|Immune system disorders|Infections and infestations|Injury, poisoning and procedural complications|Investigations|Musculoskeletal and connective tissue disorders|Neoplasms benign, malignant and unspecified (incl cysts and polyps)|Nervous system disorders|Pregnancy, puerperium and perinatal conditions|Renal and urinary disorders|Reproductive system and breast disorders|Respiratory, thoracic and mediastinal disorders|Surgical and medical procedures|Vascular disorders|Ear and labyrinth disorders|General disorders and administration site conditions|Metabolism and nutrition disorders|Product issues|Psychiatric disorders|Skin and subcutaneous tissue disorders|Social circumstances)',
           re.UNICODE)

In [31]:
def extract_table_rows(df):
    rows = []

    for row in df.itertuples():
        # First row contains the header
        if row[0] == 0:
            # Find the order of columns
            if "reaction".lower() in row[2].lower() else False:
                AD_index, F_index  = 2, 3
            else:
                AD_index, F_index  = 3, 2

        else:
            ad_reaction = row[AD_index].split('\n')
            freq = row[F_index].split('\n')
            
            for index, ad in enumerate(ad_reaction):
                ad_row = []
                ad_row.insert(0, row[1].replace('\n', ''))  
                ad_row.insert(1, ad_reaction[index].strip())
                ad_row.insert(2, freq[index].strip())
                rows.append(ad_row)
    return rows

SyntaxError: invalid syntax (2518693311.py, line 8)

In [32]:
# List all pdfs in the directory
pdfs_processed = []
pdfs_not_processed = []

for file in os.listdir(os.path.join(PATH, 'pdfs')): # ['abecma-epar-product-information_en.pdf']:
    
    if file.lower().endswith('.pdf'):
        filename = file.split('/')[-1].split('-')[0]
        fullpath = os.path.join(PATH, 'pdfs', file)
        
        print("Start Processing Filename: ", fullpath)
        
        tables = camelot.read_pdf(fullpath, pages='all', flavor='lattice', line_scale=80, shift_text = ['', 'l'], process_background=False)

        rows = []
        for i, table in enumerate(tables):
            # Convert table to a pandas DataFrame
            df = table.df
    
            # Search for regex in the first column
            matches = df[0].str.contains(regex, regex=True)
    
            # Check if any value in the column matches the regex pattern
            contains_regex = matches.any()

            if contains_regex:
                if len(df.columns) == 3:
                    # Case A: Dataframe consists of 3 columns
                    print("The file ", filename, " does have three columns table")
                    extracted_rows = extract_table_rows(df)
                    for row in extracted_rows:
                        rows.append(row)

                    pdfs_processed.append(fullpath)
                else:
                    print("The file ", filename, " does NOT have three columns table")
                    pdfs_not_processed.append(fullpath)
        if rows:
            pd.DataFrame(rows, columns=["SOC", "ADR", "FREQ"]).to_csv(os.path.join(PATH, "extracted_tables", filename + ".csv"), sep=';', encoding='utf-8', index=False)
            
# Create report

Start Processing Filename:  ./data/pdfs/metformin-hydrochloride-accord-epar-product-information_en.pdf


/var/folders/h3/2hsc1tt90kn6k_0wsgjj18nc0000gn/T/ipykernel_29736/3426297550.py:21: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  matches = df[0].str.contains(regex, regex=True)


The file  metformin  does NOT have three columns table
The file  metformin  does NOT have three columns table
Start Processing Filename:  ./data/pdfs/prehevbri-epar-product-information_en.pdf
The file  prehevbri  does have three columns table


/var/folders/h3/2hsc1tt90kn6k_0wsgjj18nc0000gn/T/ipykernel_29736/3426297550.py:21: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  matches = df[0].str.contains(regex, regex=True)


IndexError: list index out of range

In [33]:
file = "./data/pdfs/prehevbri-epar-product-information_en.pdf"

In [34]:
tables = camelot.read_pdf(fullpath, pages='all', flavor='lattice', line_scale=40, shift_text = ['', 'l'], process_background=True)

In [35]:
for i, table in enumerate(tables):
    df = table.df
    
    # Search for regex in the first column
    matches = df[0].str.contains(regex, regex=True)
    
    # Check if any value in the column matches the regex pattern
    contains_regex = matches.any()

    if contains_regex:
        if len(df.columns) == 3:
            print("Table index: ", i)

Table index:  0


/var/folders/h3/2hsc1tt90kn6k_0wsgjj18nc0000gn/T/ipykernel_29736/2842267533.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  matches = df[0].str.contains(regex, regex=True)


In [39]:
tables[0].df.head(11)

,0,1,2
0,System Organ Class,Adverse Reaction,Frequency
1,Blood and Lymphatic System \nDisorders,Lymphadenopathy,Uncommon
2,,"Diarrhoea1, nausea/vomiting1",Common
3,Gastrointestinal Disorders,Abdominal pain,Common
4,,"Injection site pain1, injection site tendernes...",Very Common
5,General Disorders and,"Injection site swelling1, injection site redness1",Common
6,Administration Site Conditions,Injection site bruising,Common
7,,Fever1,Common
8,,Headache1,Very Common
9,Nervous System Disorders,Dizziness,Common


In [14]:
rows = []

for row in df.itertuples():
    # First row contains the header
    if row[0] == 0:
        # Find the order of columns
        if "reaction".lower() in row[2].lower() else False:
            AD_index, F_index  = 2, 3
        else:
            AD_index, F_index  = 3, 2

        else:
            ad_reaction = row[AD_index].split('\n')
            freq = row[F_index].split('\n')
            
            for index, ad in enumerate(ad_reaction):
                ad_row = []
                ad_row.insert(0, row[1].replace('\n', ''))  
                ad_row.insert(1, ad_reaction[index].strip())
                ad_row.insert(2, freq[index].strip())
                rows.append(ad_row)

In [17]:
os.path.join(PATH, 'pdfs', 'abecma-epar-product-information_en.pdf')

'./data/pdfs/abecma-epar-product-information_en.pdf'

In [22]:
import PyPDF2

In [23]:
PyPDF2.__version__

'2.12.1'

In [44]:
!pip install "camelot-py[plot]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 4.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.2/258.2 kB 3.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 4.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 4.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 2.5 MB/s eta 0:00:0000:01


In [52]:
camelot.plot(tables[0], kind='text').show()

ImportError: matplotlib is required for plotting.

In [46]:
!pip install matplotlib

In [48]:
import matplotlib

In [51]:
import matplotlib
matplotlib.__version__

'3.8.2'